# Explore here

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")


In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [4]:
df_raw.sample(10)

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
1837,36023,47823,4931,10.310938,7171,14.994877,8612,18.008071,5031,10.520043,...,3149,9.9,9.1,10.8,3838,2.8,2.6,3.0,1078,5
1607,30023,9131,670,7.337641,861,9.429416,1105,12.101632,990,10.842186,...,732,13.1,12.0,14.2,1025,3.8,3.5,4.1,294,6
1593,29227,2040,227,11.127451,232,11.372549,203,9.950980,202,9.901961,...,179,13.5,12.3,14.7,219,3.7,3.4,4.1,61,6
979,20187,1987,289,14.544540,301,15.148465,206,10.367388,230,11.575239,...,111,12.9,11.9,13.8,189,3.3,3.0,3.5,48,6
1967,37159,141262,16360,11.581317,18443,13.055882,18057,12.782631,16476,11.663434,...,10221,12.9,12.0,13.8,14124,3.4,3.2,3.7,3783,2
2483,47115,28575,3189,11.160105,3424,11.982502,3283,11.489064,3137,10.978128,...,2752,14.6,13.5,15.7,3293,3.6,3.3,3.9,814,3
2356,45085,106512,14164,13.298032,13865,13.017313,16252,15.258375,13137,12.333822,...,6894,14.8,14.0,15.7,12023,3.7,3.5,4.0,3029,4
2201,40145,80110,10319,12.881039,10923,13.635002,9293,11.600300,10807,13.490201,...,5179,12.4,11.5,13.2,7489,3.1,2.9,3.3,1857,3
699,18009,11930,1382,11.584241,1465,12.279966,1338,11.215423,1280,10.729254,...,1073,14.1,13.0,15.1,1314,3.5,3.3,3.8,329,6
2951,53001,19759,4043,20.461562,3548,17.956374,2610,13.209171,2366,11.974290,...,981,12.8,12.1,13.6,1633,3.6,3.4,3.8,458,5


In [5]:
df_raw.to_csv("../data/raw/data_raw.csv")

In [6]:
df_processed = df_raw.copy()

In [8]:
column_list = df_processed.columns.to_list()

column_list

['fips',
 'TOT_POP',
 '0-9',
 '0-9 y/o % of total pop',
 '19-Oct',
 '10-19 y/o % of total pop',
 '20-29',
 '20-29 y/o % of total pop',
 '30-39',
 '30-39 y/o % of total pop',
 '40-49',
 '40-49 y/o % of total pop',
 '50-59',
 '50-59 y/o % of total pop',
 '60-69',
 '60-69 y/o % of total pop',
 '70-79',
 '70-79 y/o % of total pop',
 '80+',
 '80+ y/o % of total pop',
 'White-alone pop',
 '% White-alone',
 'Black-alone pop',
 '% Black-alone',
 'Native American/American Indian-alone pop',
 '% NA/AI-alone',
 'Asian-alone pop',
 '% Asian-alone',
 'Hawaiian/Pacific Islander-alone pop',
 '% Hawaiian/PI-alone',
 'Two or more races pop',
 '% Two or more races',
 'POP_ESTIMATE_2018',
 'N_POP_CHG_2018',
 'GQ_ESTIMATES_2018',
 'R_birth_2018',
 'R_death_2018',
 'R_NATURAL_INC_2018',
 'R_INTERNATIONAL_MIG_2018',
 'R_DOMESTIC_MIG_2018',
 'R_NET_MIG_2018',
 'Less than a high school diploma 2014-18',
 'High school diploma only 2014-18',
 "Some college or associate's degree 2014-18",
 "Bachelor's degree or 

In [11]:
#Se seleccionan las variables independientes que podrían aportar más informacion todas ellas numericas

df_interin = df_processed[["0-9","19-Oct","20-29","30-39","40-49","50-59","60-69","70-79","80+","% White-alone","% Black-alone","% NA/AI-alone","% Asian-alone","% Hawaiian/PI-alone","Less than a high school diploma 2014-18","High school diploma only 2014-18","Bachelor's degree or higher 2014-18","MEDHHINC_2018","Employed_2018","Unemployed_2018","Active Physicians per 100000 Population 2018 (AAMC)","Obesity_prevalence","diabetes_prevalence","CKD_prevalence","COPD_prevalence","POVALL_2018","Heart disease_prevalence"]]

In [10]:
df_interin.describe().T

,count,mean,std,min,25%,50%,75%,max
0-9,3140.0,12740.302866,41807.301846,0.000000,1280.500000,3057.000000,8097.000000,1.208253e+06
19-Oct,3140.0,13367.976752,42284.392134,0.000000,1374.500000,3274.000000,8822.250000,1.239139e+06
20-29,3140.0,14469.331529,49577.725396,0.000000,1263.750000,3108.000000,8976.250000,1.557073e+06
30-39,3140.0,13916.490127,48990.951710,11.000000,1232.750000,3000.500000,8314.250000,1.501844e+06
40-49,3140.0,12885.214331,43472.189993,4.000000,1245.250000,3048.500000,8120.500000,1.350076e+06
50-59,3140.0,13638.031529,42720.629434,14.000000,1483.000000,3515.500000,9157.250000,1.306003e+06
60-69,3140.0,11916.213057,34518.581878,20.000000,1476.500000,3410.000000,8710.500000,1.014325e+06
70-79,3140.0,7220.310510,20287.498429,6.000000,950.750000,2177.000000,5574.000000,5.725300e+05
80+,3140.0,4035.541720,12063.782251,0.000000,533.000000,1160.000000,2899.750000,3.562750e+05
% White-alone,3140.0,84.494109,16.348987,3.841985,79.620473,91.410189,95.521727,9.904378e+01


In [13]:
df_interin.select_dtypes('number').corr().style.background_gradient(cmap='Blues')

,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,% White-alone,% Black-alone,% NA/AI-alone,% Asian-alone,% Hawaiian/PI-alone,Less than a high school diploma 2014-18,High school diploma only 2014-18,Bachelor's degree or higher 2014-18,MEDHHINC_2018,Employed_2018,Unemployed_2018,Active Physicians per 100000 Population 2018 (AAMC),Obesity_prevalence,diabetes_prevalence,CKD_prevalence,COPD_prevalence,POVALL_2018,Heart disease_prevalence
0-9,1.000000,0.997487,0.989992,0.990971,0.991905,0.985749,0.979333,0.961958,0.946346,-0.156263,0.090312,-0.039405,0.464069,0.034929,0.946713,0.973324,0.947922,0.266013,0.988804,0.980304,0.146373,-0.233481,-0.167495,-0.216430,-0.253851,0.970050,-0.302992
19-Oct,0.997487,1.000000,0.988552,0.987298,0.992803,0.988988,0.982460,0.966966,0.951206,-0.151108,0.085954,-0.042076,0.466383,0.032371,0.941898,0.972866,0.949457,0.279487,0.990084,0.980247,0.153894,-0.244549,-0.178421,-0.226956,-0.260600,0.962676,-0.311823
20-29,0.989992,0.988552,1.000000,0.995803,0.991338,0.986506,0.981665,0.964468,0.955231,-0.158644,0.092785,-0.040374,0.467676,0.033503,0.952755,0.968700,0.961699,0.251673,0.992286,0.983038,0.159569,-0.247443,-0.175033,-0.219593,-0.253190,0.972097,-0.304290
30-39,0.990971,0.987298,0.995803,1.000000,0.995385,0.988790,0.983050,0.965318,0.955137,-0.157205,0.088565,-0.039967,0.479933,0.034177,0.948777,0.968551,0.970641,0.268507,0.995252,0.981297,0.157943,-0.247318,-0.170921,-0.215739,-0.251209,0.965289,-0.298536
40-49,0.991905,0.992803,0.991338,0.995385,1.000000,0.996229,0.989045,0.974370,0.963603,-0.150962,0.083884,-0.042980,0.479834,0.032044,0.949198,0.973548,0.969192,0.283480,0.997418,0.983027,0.158844,-0.250376,-0.171510,-0.217762,-0.249727,0.958807,-0.297484
50-59,0.985749,0.988988,0.986506,0.988790,0.996229,1.000000,0.996358,0.985032,0.977812,-0.151358,0.086051,-0.046412,0.480258,0.030808,0.940256,0.981409,0.969900,0.293244,0.996251,0.980728,0.184654,-0.260781,-0.177161,-0.220956,-0.251045,0.952911,-0.299648
60-69,0.979333,0.982460,0.981665,0.983050,0.989045,0.996358,1.000000,0.993717,0.986548,-0.150112,0.085311,-0.048916,0.478807,0.033664,0.927388,0.984756,0.967485,0.289337,0.991836,0.976082,0.195129,-0.272592,-0.180107,-0.217976,-0.249446,0.950705,-0.296324
70-79,0.961958,0.966966,0.964468,0.965318,0.974370,0.985032,0.993717,1.000000,0.993476,-0.142063,0.079386,-0.050415,0.467874,0.033997,0.910758,0.978348,0.953889,0.283130,0.977661,0.961170,0.196528,-0.278248,-0.171794,-0.202424,-0.236959,0.935693,-0.277776
80+,0.946346,0.951206,0.955231,0.955137,0.963603,0.977812,0.986548,0.993476,1.000000,-0.137237,0.074990,-0.049353,0.463761,0.033103,0.909713,0.970545,0.951754,0.271834,0.968502,0.953036,0.206679,-0.272228,-0.165779,-0.192883,-0.230521,0.927030,-0.266539
% White-alone,-0.156263,-0.151108,-0.158644,-0.157205,-0.150962,-0.151358,-0.150112,-0.142063,-0.137237,1.000000,-0.828582,-0.402107,-0.246722,-0.120163,-0.128063,-0.153390,-0.158356,0.162865,-0.150249,-0.146545,0.106708,-0.238957,-0.410392,-0.308305,-0.056862,-0.167664,-0.011185


In [ ]:

X = df_interin.drop(["Heart disease_prevalence"], axis= 1)
y = df_interin["Heart disease_prevalence"]